# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [1]:
%%bash

source ~/.bash_profile

#### Check environment variables are set with valid values
echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

#### Download the `kfctl` CLI tool
curl --location https://github.com/kubeflow/kfctl/releases/download/v1.0.2/kfctl_v1.0.2-0-ga476281_linux.tar.gz | tar xz
sudo mv kfctl /usr/local/bin

#### Get the latest Kubeflow configuration file
export CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml"
echo "export CONFIG_URI=${CONFIG_URI}" | tee -a ~/.bash_profile

#### Set Kubeflow environment variables 
export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

#### Customize the configuration files
# We'll edit the configuration with the right names for the cluster and node groups before deploying Kubeflow.

mkdir -p ${KF_DIR}
cd ${KF_DIR}

curl -O ${CONFIG_URI}

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

#### Generate the Kubeflow installation files
cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

#### Deploy Kubeflow
cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

#### Delete the usage reporting beacon
kubectl delete all -l app=spartakus --namespace=kubeflow

#### Change to `kubeflow` namespace
kubectl config set-context --current --namespace=kubeflow

sagemaker-us-west-2-250107111215
cluster
eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-1UY17I42EZJQJ
arn:aws:iam::250107111215:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-1UY17I42EZJQJ
export CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml
export KF_NAME=cluster
export KF_DIR=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster
export CONFIG_FILE=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml
customresourcedefinition.apiextensions.k8s.io/adapters.config.istio.io unchanged
customresourcedefinition.apiextensions.k8s.io/apikeys.config.istio.io unchanged
customresourcedefinition.apiextensions.k8s.io/attributemanifests.config.istio.io unchanged
customresourcedefinition.apiextensions.k8s.io/authorizations.config.istio.io unchanged
customresourcedefinition.apiextensions.k8s.io/bypasses.config.istio.io unchanged
customresourcedefinition.apiextensions.k8s.io/certificates.certmanager.k8s.io unc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   649    0     0   1978      0 --:--:-- --:--:-- --:--:--  1972
100 27.9M  100 27.9M    0     0  11.7M      0  0:00:02  0:00:02 --:--:-- 20.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8801  100  8801    0     0   179k      0 --:--:-- --:--:-- --:--:--  182k
time="2020-08-22T21:09:37Z" level=info msg="No name specified in KfDef.Metadata.Name; defaulting to cluster based on location of config file: /home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml." filename="coordinator/coordinator.go:202"
time="2020-08-22T21:09:37Z" level=info msg="Running `eksctl version` ..." filename="utils/awsutil.go:71"
time="2020-08-22T21:09:37Z" level=info msg="eksctl version: 0.26.0" filename="ut

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [2]:
!kubectl -n kubeflow get all


NAME                                                               READY   STATUS              RESTARTS   AGE
pod/admission-webhook-bootstrap-stateful-set-0                     0/1     ContainerCreating   0          44s
pod/admission-webhook-deployment-59bc556b94-lc5vl                  0/1     ContainerCreating   0          44s
pod/alb-ingress-controller-798fbd8c96-76vhc                        0/1     ContainerCreating   0          4s
pod/application-controller-stateful-set-0                          1/1     Running             0          73s
pod/argo-ui-5f845464d7-kgbcv                                       1/1     Running             0          45s
pod/centraldashboard-d5c6d6bf-5cwwm                                0/1     ContainerCreating   0          45s
pod/jupyter-web-app-deployment-544b7d5684-7cb8t                    1/1     Running             0          44s
pod/katib-controller-6b87947df8-q2lfz                              1/1     Running             1          7s
pod/katib-db

In [3]:
!kubectl -n istio-system get all


NAME                                          READY   STATUS              RESTARTS   AGE
pod/cluster-local-gateway-78f6cbff8d-scdvr    1/1     Running             0          76s
pod/grafana-68bcfd88b6-zxcp4                  1/1     Running             0          78s
pod/istio-citadel-7dd6877d4d-crggc            1/1     Running             0          78s
pod/istio-cleanup-secrets-1.1.6-cr8mb         0/1     Completed           0          77s
pod/istio-egressgateway-7c888bd9b9-94c7f      1/1     Running             0          78s
pod/istio-galley-5bc58d7c89-s65f5             1/1     Running             0          78s
pod/istio-grafana-post-install-1.1.6-rmg5s    0/1     Completed           0          77s
pod/istio-ingressgateway-866fb99878-frk4t     1/1     Running             0          78s
pod/istio-pilot-67f9bd57b-p5f2g               2/2     Running             0          78s
pod/istio-policy-749ff546dd-g9tqf             2/2     Running             2          78s
pod/istio-security-po

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>